In [1]:
# train_transformer.py

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,  roc_auc_score

from tqdm import tqdm
import numpy as np

# ----------------------------
# 1️⃣ Load & clean dataset
# ----------------------------
df = pd.read_csv(r"R:\AIniverse\PROJECTS\mainstream\senti\data\train.csv\train.csv")  # path to your CSV
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def clean_text(text):
    text = str(text)
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['comment_text'] = df['comment_text'].apply(clean_text)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

all_text = list(train_df['comment_text'])
words = list(set(word for sent in all_text for word in sent.split()))
vocab = {w:i+2 for i,w in enumerate(words)}  # 0:PAD, 1:UNK
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1
vocab_size = len(vocab)

def encode_text(text, max_len=128):
    tokens = [vocab.get(w,1) for w in text.split()]
    if len(tokens) < max_len:
        tokens += [0]*(max_len-len(tokens))
    else:
        tokens = tokens[:max_len]
    return tokens

class ToxicDataset(Dataset):
    def __init__(self, df):
        self.texts = [encode_text(x) for x in df['comment_text']]
        self.labels = torch.tensor(df[label_cols].values, dtype=torch.float32)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {'input_ids': torch.tensor(self.texts[idx], dtype=torch.long),
                'labels': self.labels[idx]}

train_dataset = ToxicDataset(train_df)
val_dataset = ToxicDataset(val_df)
test_dataset = ToxicDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


class MiniTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, n_heads=4, ff_dim=256, num_layers=2, output_dim=6, max_len=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=n_heads, dim_feedforward=ff_dim, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(embed_dim, output_dim)
        self.max_len = max_len

    def forward(self, x):
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = self.transformer(x)
        x = self.dropout(x[:,0])  # take first token for classification
        return self.fc(x)

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MiniTransformer(vocab_size).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3)

epochs = 20
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss/len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss/len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Val Loss {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "mini_transformer_toxic_model.pt")
        print("✅ Model improved. Saved.")
    else:
        early_stop_counter +=1
        if early_stop_counter>=patience:
            print("⛔ Earl   y stopping triggered.")
            break

100%|██████████| 3990/3990 [01:35<00:00, 41.67it/s, loss=0.521] 


Epoch 1: Train Loss 0.1367, Val Loss 0.1268
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:35<00:00, 41.70it/s, loss=0.0446]


Epoch 2: Train Loss 0.1289, Val Loss 0.1197
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:35<00:00, 41.69it/s, loss=0.0687]


...

Epoch 18: Train Loss 0.0566, Val Loss 0.0645
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:40<00:00, 39.71it/s, loss=0.0137] 


Epoch 19: Train Loss 0.0539, Val Loss 0.0662


100%|██████████| 3990/3990 [01:46<00:00, 37.53it/s, loss=0.00137]


Epoch 20: Train Loss 0.0532, Val Loss 0.0677


In [3]:

# ----------------------------
# 6️⃣ Evaluation
# ----------------------------
model.load_state_dict(torch.load("mini_transformer_toxic_model.pt"))
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = torch.sigmoid(model(input_ids)).cpu().numpy()
        preds = (outputs>0.5).astype(int)
        all_labels.append(labels)
        all_preds.append(preds)

all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

for i,col in enumerate(label_cols):
    print(f"--- {col} ---")
    print("Accuracy: ", (all_labels[:,i]==all_preds[:,i]).mean())
    print("F1-score:", f1_score(all_labels[:,i], all_preds[:,i]))
    print("ROC-AUC:", roc_auc_score(all_labels[:,i], all_preds[:,i]))

C:\Users\anmol\AppData\Local\Temp\ipykernel_7016\2318801401.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mini_transformer_toxic_mode

--- toxic ---
Accuracy:  0.9572001503947863
F1-score: 0.7454342154304883
ROC-AUC: 0.8233025422678458
--- severe_toxic ---
Accuracy:  0.9891590424865271
F1-score: 0.4175084175084175
ROC-AUC: 0.689047313118808
--- obscene ---
Accuracy:  0.9754981827296654
F1-score: 0.7605633802816901
ROC-AUC: 0.8575687660358162
--- threat ---
Accuracy:  0.9976814137109914
F1-score: 0.0
ROC-AUC: 0.5
--- insult ---
Accuracy:  0.9679157789196641
F1-score: 0.6679636835278858
ROC-AUC: 0.811460396039604
--- identity_hate ---
Accuracy:  0.9913522997869407
F1-score: 0.0
ROC-AUC: 0.5


In [ ]:
# transformer_user_test.py

import torch
import torch.nn as nn
import re
import html
import numpy as np

vocab = torch.load("transformer_vocab.pt")  # saved vocab
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
max_len = 128

def clean_text(text):
    text = str(text)
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def encode_text(text):
    tokens = [vocab.get(w,1) for w in text.split()]
    if len(tokens)<max_len:
        tokens += [0]*(max_len-len(tokens))
    else:
        tokens = tokens[:max_len]
    return torch.tensor(tokens).unsqueeze(0)

class MiniTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, n_heads=4, ff_dim=256, num_layers=2, output_dim=6, max_len=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Embedding(max_len, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=n_heads, dim_feedforward=ff_dim, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(embed_dim, output_dim)
        self.max_len = max_len

    def forward(self, x):
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0)
        x = self.embedding(x) + self.pos_embedding(positions)
        x = self.transformer(x)
        x = self.dropout(x[:,0])
        return self.fc(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MiniTransformer(vocab_size=len(vocab)).to(device)
model.load_state_dict(torch.load("mini_transformer_toxic_model.pt", map_location=device))
model.eval()

def predict_toxicity(comment):
    comment = clean_text(comment)
    input_ids = encode_text(comment).to(device)
    with torch.no_grad():
        logits = model(input_ids)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    result = {label_cols[i]: {'prob': float(probs[i]), 'pred': int(probs[i]>0.5)} for i in range(len(label_cols))}
    return result

while True:
    text = input("Enter comment (or 'quit' to exit): ")
    if text.lower() == 'quit':
        break
    output = predict_toxicity(text)
    print(output)
